# 추천 시스템 - Content based filtering

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
product = pd.read_csv('../dataset/preprocessing_product.csv')
product

,Unnamed: 0,product_id,product_name,product_price,food_ingredient
0,0,1,[fresheasy] 나만의 심야식당 3종 세트,"26,800원","['기타가공품', '건마늘분말', '폴리인산나트륨', '복합조미식품', '치자적색소..."
1,1,2,[fresheasy] 누구나홀딱반한닭 바사칸치킨윙봉 400g 1팩,"32,700원","['기타가공품', '복합조미식품', '식물성유지', '닭고기', '혼합제제', '정..."
2,2,3,[fresheasy] 누구나홀딱반한닭 바사칸치킨윙봉 400g 3팩,"32,700원","['기타가공품', '복합조미식품', '식물성유지', '닭고기', '혼합제제', '정..."
3,3,4,[fresheasy] 누구나홀딱반한닭 바사칸치킨윙봉400g 3팩,"21,900원","['기타가공품', '복합조미식품', '식물성유지', '닭고기', '혼합제제', '정..."
4,4,5,[fresheasy] 누구나홀딱반한닭 쉬림프치킨파히타 2인분 3팩,"53,700원","['기타 수산물가공품', '복합조미식품', '빵류', '청피망', '자연치즈', '..."
...,...,...,...,...,...
304,304,305,풀무원 전골 밀키트 4종(만두전골/어묵전골) 2개 골라담기,"25,130원","['만두', '숙면', '혼합야채', '어묵', '소고기', '소스']"
305,305,306,프레시지 사천식 마라탕 2인분,"15,900원","['소시지', '배추', '팽이버섯', '향미유', '소고기', '소스', '유바'..."
306,306,307,프레시지 우삼겹 마라 떡볶이 2인분,"15,900원","['소시지', '어묵', '향미유', '소고기', '떡류', '소스', '건면']"
307,307,308,핑크퐁 아기상어 클레이 쿠키 만들기 250g x 2개,"20,490원","['기타가공품', '정제소금', '설탕', '난황액', '밀가루', '코코아분말',..."


In [4]:
product.columns

Index(['Unnamed: 0', 'product_id', 'product_name', 'product_price',
       'food_ingredient'],
      dtype='object')

In [5]:
product = product.drop('Unnamed: 0',axis=1)
product

,product_id,product_name,product_price,food_ingredient
0,1,[fresheasy] 나만의 심야식당 3종 세트,"26,800원","['기타가공품', '건마늘분말', '폴리인산나트륨', '복합조미식품', '치자적색소..."
1,2,[fresheasy] 누구나홀딱반한닭 바사칸치킨윙봉 400g 1팩,"32,700원","['기타가공품', '복합조미식품', '식물성유지', '닭고기', '혼합제제', '정..."
2,3,[fresheasy] 누구나홀딱반한닭 바사칸치킨윙봉 400g 3팩,"32,700원","['기타가공품', '복합조미식품', '식물성유지', '닭고기', '혼합제제', '정..."
3,4,[fresheasy] 누구나홀딱반한닭 바사칸치킨윙봉400g 3팩,"21,900원","['기타가공품', '복합조미식품', '식물성유지', '닭고기', '혼합제제', '정..."
4,5,[fresheasy] 누구나홀딱반한닭 쉬림프치킨파히타 2인분 3팩,"53,700원","['기타 수산물가공품', '복합조미식품', '빵류', '청피망', '자연치즈', '..."
...,...,...,...,...
304,305,풀무원 전골 밀키트 4종(만두전골/어묵전골) 2개 골라담기,"25,130원","['만두', '숙면', '혼합야채', '어묵', '소고기', '소스']"
305,306,프레시지 사천식 마라탕 2인분,"15,900원","['소시지', '배추', '팽이버섯', '향미유', '소고기', '소스', '유바'..."
306,307,프레시지 우삼겹 마라 떡볶이 2인분,"15,900원","['소시지', '어묵', '향미유', '소고기', '떡류', '소스', '건면']"
307,308,핑크퐁 아기상어 클레이 쿠키 만들기 250g x 2개,"20,490원","['기타가공품', '정제소금', '설탕', '난황액', '밀가루', '코코아분말',..."


In [8]:
orders = pd.read_csv('../dataset/preprocessing_orders.csv')
orders = orders.drop('Unnamed: 0',axis=1)
orders

,order_id,order_date,product_id,member_id
0,1,2024-02-01 05:45:13.398265,87,7260
1,2,2024-01-28 19:50:16.218157,161,1340
2,3,2024-01-08 07:49:56.067986,86,392
3,4,2024-01-08 08:39:25.103003,277,5690
4,5,2024-02-19 01:30:09.341400,87,7823
...,...,...,...,...
9995,9996,2024-01-29 15:28:21.989823,122,6586
9996,9997,2024-02-15 06:11:18.692664,39,3001
9997,9998,2024-02-13 10:06:34.004773,122,8738
9998,9999,2024-01-07 14:46:59.073925,130,1446


In [9]:
member = pd.read_csv('../dataset/preprocessing_member.csv')
member = member.drop('Unnamed: 0',axis=1)
member

,member_id,member_email,member_name,member_nickname,member_phone,member_gender,member_birth,member_role,created_at,modified_at,deleted_at,refresh_token
0,1046,gyeongsug68@gmail.com,장현주,gimyeongjin,010-2359-3069,Male,1999-01-23,user,2024-02-19 05:41:45.896441,2024-01-26 07:50:55.482923,2024-01-03 06:10:17.162867,4ddaba04ae25f9a10bfbeadebd5f1d01c50eca49baab25...
1,676,fha@gmail.com,박현정,bagjieun,010-8470-3845,Male,2001-09-28,user,2024-01-12 02:50:42.526686,2024-01-16 22:50:02.847855,NaN,bf98a3f36305961814a4d2f068f7636991bef5147ea68c...
2,5021,hyeonjii@live.com,김영수,ujin62,010-2022-8267,Male,2005-08-22,user,2024-01-13 07:27:05.256302,2024-01-09 19:49:10.410905,NaN,edd81879a07f07259be4368929ea5dc5225deeeed7906d...
3,8813,yeongsigi@hotmail.com,박정수,seunghyeoncoe,010-3043-1144,Male,2005-08-31,user,2024-01-24 16:41:21.445683,2024-01-24 20:42:09.681452,NaN,c93328ccb1142fd1fdb69c0fdcd81cf20a4dc1e524f973...
4,3347,xu@nate.com,홍하윤,jongsu17,010-3531-8749,Male,2003-02-28,user,2024-02-20 00:56:40.045554,2024-02-09 15:06:05.743431,NaN,383ed9c51cb7081a49c5df6167862ac4dadc9262f69e3e...
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,doyun93@gmail.com,김동현,cgim,010-5897-5822,Male,2004-08-24,user,2024-02-07 10:45:47.980599,2024-01-31 05:17:32.958590,2024-01-16 02:15:38.315181,9e2cb7eab30b9d96f48378d9b2b237327d2d0f4e5df35d...
9996,9997,jihu49@hanmail.net,박예진,sanghunseo,010-6839-6387,Male,2003-12-03,user,2024-02-11 17:27:11.186506,2024-01-04 16:16:54.024429,NaN,7b05e2fdefbdfee579bee7b8a3378ea62a16d3aef98b0d...
9997,9998,yeongceol29@live.com,권경숙,seohyeon50,010-9496-4657,Female,2000-12-12,user,2024-01-01 19:41:10.521513,2024-02-11 10:23:40.255759,2024-01-01 13:57:13.178545,acaf7a9d248f9514d6f7d0f8cd4ba8d96f3d3429f50ea7...
9998,9999,lbae@live.com,김예준,seonghyeon58,010-4614-4616,Male,2002-02-22,user,2024-01-29 23:42:47.511431,2024-01-31 23:26:49.448482,NaN,cd3cc7b5372e776ee5ce19e93921fa2d6c620b06f3e431...


In [10]:
product

,product_id,product_name,product_price,food_ingredient
0,1,[fresheasy] 나만의 심야식당 3종 세트,"26,800원","['기타가공품', '건마늘분말', '폴리인산나트륨', '복합조미식품', '치자적색소..."
1,2,[fresheasy] 누구나홀딱반한닭 바사칸치킨윙봉 400g 1팩,"32,700원","['기타가공품', '복합조미식품', '식물성유지', '닭고기', '혼합제제', '정..."
2,3,[fresheasy] 누구나홀딱반한닭 바사칸치킨윙봉 400g 3팩,"32,700원","['기타가공품', '복합조미식품', '식물성유지', '닭고기', '혼합제제', '정..."
3,4,[fresheasy] 누구나홀딱반한닭 바사칸치킨윙봉400g 3팩,"21,900원","['기타가공품', '복합조미식품', '식물성유지', '닭고기', '혼합제제', '정..."
4,5,[fresheasy] 누구나홀딱반한닭 쉬림프치킨파히타 2인분 3팩,"53,700원","['기타 수산물가공품', '복합조미식품', '빵류', '청피망', '자연치즈', '..."
...,...,...,...,...
304,305,풀무원 전골 밀키트 4종(만두전골/어묵전골) 2개 골라담기,"25,130원","['만두', '숙면', '혼합야채', '어묵', '소고기', '소스']"
305,306,프레시지 사천식 마라탕 2인분,"15,900원","['소시지', '배추', '팽이버섯', '향미유', '소고기', '소스', '유바'..."
306,307,프레시지 우삼겹 마라 떡볶이 2인분,"15,900원","['소시지', '어묵', '향미유', '소고기', '떡류', '소스', '건면']"
307,308,핑크퐁 아기상어 클레이 쿠키 만들기 250g x 2개,"20,490원","['기타가공품', '정제소금', '설탕', '난황액', '밀가루', '코코아분말',..."


In [11]:
tfidf_vector = TfidfVectorizer()

In [12]:
tfidf_matrix = tfidf_vector.fit_transform(product['food_ingredient']).toarray()

In [13]:
tfidf_matrix_feature = tfidf_vector.get_feature_names_out()

In [14]:
tfidf_matrix.shape

(309, 351)

In [15]:
tfidf_matrix = pd.DataFrame(tfidf_matrix, columns=tfidf_matrix_feature, index = product.product_name)
print(tfidf_matrix.shape)
tfidf_matrix.head()

(309, 351)


,black,pepper,white,가공버터,가공소금,가공유크림,가공치즈,가지,간마늘,간생강,...,홍합살,황기,효모,효모식품,효모추출물,효소처리스테비아,후추가루,흰다리새우,히드록시프로필메틸셀룰로스,히드록시프로필인산이전분
product_name,,,,,,,,,,,,,,,,,,,,,
[fresheasy] 나만의 심야식당 3종 세트,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
[fresheasy] 누구나홀딱반한닭 바사칸치킨윙봉 400g 1팩,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
[fresheasy] 누구나홀딱반한닭 바사칸치킨윙봉 400g 3팩,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
[fresheasy] 누구나홀딱반한닭 바사칸치킨윙봉400g 3팩,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
[fresheasy] 누구나홀딱반한닭 쉬림프치킨파히타 2인분 3팩,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
import pickle

with open("tfidf_vector.pkl", "wb") as f:
    pickle.dump(tfidf_vector, f)

In [17]:
%%time
cosine_sim = cosine_similarity(tfidf_matrix)

CPU times: user 129 ms, sys: 3.77 ms, total: 133 ms
Wall time: 35.2 ms


In [18]:
cosine_sim.shape

(309, 309)

In [19]:
cosine_sim_df = pd.DataFrame(cosine_sim, index = product.product_name, columns = product.product_name)
print(cosine_sim_df.shape)
cosine_sim_df.head()

(309, 309)


product_name,[fresheasy] 나만의 심야식당 3종 세트,[fresheasy] 누구나홀딱반한닭 바사칸치킨윙봉 400g 1팩,[fresheasy] 누구나홀딱반한닭 바사칸치킨윙봉 400g 3팩,[fresheasy] 누구나홀딱반한닭 바사칸치킨윙봉400g 3팩,[fresheasy] 누구나홀딱반한닭 쉬림프치킨파히타 2인분 3팩,[fresheasy] 누구나홀딱반한닭 치킨투움바파스타 (384g) 2인분 3팩,[fresheasy] 누구나홀딱반한닭 치킨투움바파스타 2인분 3팩,[fresheasy] 누구나홀딱반할 야식 4종 세트,[fresheasy] 백년가게 도가니탕 600g 1팩,[fresheasy] 백년가게 도가니탕 600g 5팩 (상온),...,풀무원 반듯한식 건강한 부대찌개 460g x 10봉,풀무원 반듯한식 건강한 부대찌개 460g x 5봉,풀무원 반듯한식 건강한 부대찌개 460g x 7봉,풀무원 쌀 국물떡볶이 423.5g(2인분) x 4봉,풀무원 우삼겹 어묵전골 밀키트 2종(버섯/등촌식) 2개 골라담기,풀무원 전골 밀키트 4종(만두전골/어묵전골) 2개 골라담기,프레시지 사천식 마라탕 2인분,프레시지 우삼겹 마라 떡볶이 2인분,핑크퐁 아기상어 클레이 쿠키 만들기 250g x 2개,핑크퐁 아기상어 클레이 쿠키 만들기 250g x 4개
product_name,,,,,,,,,,,,,,,,,,,,,
[fresheasy] 나만의 심야식당 3종 세트,1.000000,0.607387,0.607387,0.607387,0.064986,0.099104,0.099104,0.907259,0.030423,0.030423,...,0.035574,0.035574,0.035574,0.070198,0.019953,0.017560,0.013485,0.017071,0.147593,0.147593
[fresheasy] 누구나홀딱반한닭 바사칸치킨윙봉 400g 1팩,0.607387,1.000000,1.000000,1.000000,0.036949,0.113168,0.113168,0.551057,0.050088,0.050088,...,0.041891,0.041891,0.041891,0.115573,0.000000,0.000000,0.000000,0.000000,0.051724,0.051724
[fresheasy] 누구나홀딱반한닭 바사칸치킨윙봉 400g 3팩,0.607387,1.000000,1.000000,1.000000,0.036949,0.113168,0.113168,0.551057,0.050088,0.050088,...,0.041891,0.041891,0.041891,0.115573,0.000000,0.000000,0.000000,0.000000,0.051724,0.051724
[fresheasy] 누구나홀딱반한닭 바사칸치킨윙봉400g 3팩,0.607387,1.000000,1.000000,1.000000,0.036949,0.113168,0.113168,0.551057,0.050088,0.050088,...,0.041891,0.041891,0.041891,0.115573,0.000000,0.000000,0.000000,0.000000,0.051724,0.051724
[fresheasy] 누구나홀딱반한닭 쉬림프치킨파히타 2인분 3팩,0.064986,0.036949,0.036949,0.036949,1.000000,0.247202,0.247202,0.058959,0.021524,0.021524,...,0.031655,0.031655,0.031655,0.000000,0.026891,0.023666,0.018173,0.023006,0.000000,0.000000


In [20]:
def ingredient_recommendations(target_name, matrix, items, k=10):
    recom_idx = matrix.loc[:, target_name].values.reshape(1, -1).argsort()[:, ::-1].flatten()[1:k+1]
    recom_name = items.iloc[recom_idx, :].product_name.values
    recom_ingredient = items.iloc[recom_idx, :].food_ingredient.values
    target_name_list = np.full(len(range(k)), target_name)
    target_ingredient_list = np.full(len(range(k)), items[items.product_name == target_name].food_ingredient.values)
    d = {
        'target_name':target_name_list,
        'target_ingredient':target_ingredient_list,
        'recom_name' : recom_name,
        'recom_ingredient' : recom_ingredient
    }
    return pd.DataFrame(d)

In [21]:
ingredient_recommendations('[fresheasy] 누구나홀딱반한닭 쉬림프치킨파히타 2인분 3팩', cosine_sim_df, product)

,target_name,target_ingredient,recom_name,recom_ingredient
0,[fresheasy] 누구나홀딱반한닭 쉬림프치킨파히타 2인분 3팩,"['기타 수산물가공품', '복합조미식품', '빵류', '청피망', '자연치즈', '...",[fresheasy] 패밀리 양식다이닝 4종 세트,"['기타 수산물가공품', '복합조미식품', '기타가공품', '빵류', '청피망', ..."
1,[fresheasy] 누구나홀딱반한닭 쉬림프치킨파히타 2인분 3팩,"['기타 수산물가공품', '복합조미식품', '빵류', '청피망', '자연치즈', '...",[fresheasy] 홀리데이 채끝 스테이크 5종세트,"['복합조미식품', '곡류가공품', '볶은마늘분말', '빵류', '서류가공품', '..."
2,[fresheasy] 누구나홀딱반한닭 쉬림프치킨파히타 2인분 3팩,"['기타 수산물가공품', '복합조미식품', '빵류', '청피망', '자연치즈', '...",[fresheasy] 홀리데이 채끝 스테이크 5종 세트,"['복합조미식품', '곡류가공품', '볶은마늘분말', '빵류', '서류가공품', '..."
3,[fresheasy] 누구나홀딱반한닭 쉬림프치킨파히타 2인분 3팩,"['기타 수산물가공품', '복합조미식품', '빵류', '청피망', '자연치즈', '...",[모두의맛집 - 서울지짐이] 오징어비빔쫄면,"['기타 수산물가공품', '숙면', '콩나물', '소스', '당근']"
4,[fresheasy] 누구나홀딱반한닭 쉬림프치킨파히타 2인분 3팩,"['기타 수산물가공품', '복합조미식품', '빵류', '청피망', '자연치즈', '...",[모두의맛집 - 수제비먹는닭갈비] 바지락 수제비,"['기타 수산물가공품', '생면', '소스', '쥬키니호박', '대파']"
5,[fresheasy] 누구나홀딱반한닭 쉬림프치킨파히타 2인분 3팩,"['기타 수산물가공품', '복합조미식품', '빵류', '청피망', '자연치즈', '...",당일 제조하여 더욱 신선한 쿡솜씨 이탈리안감바스,"['기타 수산물가공품', '새송이버섯', '빵류', '파슬리잎', '토마토열매', ..."
6,[fresheasy] 누구나홀딱반한닭 쉬림프치킨파히타 2인분 3팩,"['기타 수산물가공품', '복합조미식품', '빵류', '청피망', '자연치즈', '...",[프레시지] 온가족 홈파티 밀키트 4종 세트(블랙라벨 2팩+감바스+쉬림프로제),"['기타 수산물가공품', '복합조미식품', '유함유가공품', '새송이버섯', '정제..."
7,[fresheasy] 누구나홀딱반한닭 쉬림프치킨파히타 2인분 3팩,"['기타 수산물가공품', '복합조미식품', '빵류', '청피망', '자연치즈', '...",삼립 시티델리 브리또 불고기라이스 600g (150g 4입) 2봉,"['곡류가공품', '복합조미식품', '기타가공품', '빵류', '설탕', '모조치즈..."
8,[fresheasy] 누구나홀딱반한닭 쉬림프치킨파히타 2인분 3팩,"['기타 수산물가공품', '복합조미식품', '빵류', '청피망', '자연치즈', '...",삼립 시티델리 이탈리아식 구운 만두 칼조네 콤비네이션 440g 2봉,"['복합조미식품', '정제소금', '설탕', '식품첨가물', '자연치즈', '소스'..."
9,[fresheasy] 누구나홀딱반한닭 쉬림프치킨파히타 2인분 3팩,"['기타 수산물가공품', '복합조미식품', '빵류', '청피망', '자연치즈', '...",당일 제조하여 더욱 신선한 쿡솜씨 골뱅이무침,"['양배추', '수산물가공품', '청양고추', '냉면', '소스', '당근', '양..."


In [22]:
ingredient_recommendations('프레시지 우삼겹 마라 떡볶이 2인분', cosine_sim_df, product)

,target_name,target_ingredient,recom_name,recom_ingredient
0,프레시지 우삼겹 마라 떡볶이 2인분,"['소시지', '어묵', '향미유', '소고기', '떡류', '소스', '건면']",프레시지 사천식 마라탕 2인분,"['소시지', '배추', '팽이버섯', '향미유', '소고기', '소스', '유바'..."
1,프레시지 우삼겹 마라 떡볶이 2인분,"['소시지', '어묵', '향미유', '소고기', '떡류', '소스', '건면']",풀무원 우삼겹 어묵전골 밀키트 2종(버섯/등촌식) 2개 골라담기,"['숙면', '혼합야채', '어묵', '소고기', '소스']"
2,프레시지 우삼겹 마라 떡볶이 2인분,"['소시지', '어묵', '향미유', '소고기', '떡류', '소스', '건면']",풀무원 떡볶이(쌀떡/밀떡/누들/고추장크림/양송이크림) 6봉 골라담기,"['떡류', '기타가공품', '소스', '소스류']"
3,프레시지 우삼겹 마라 떡볶이 2인분,"['소시지', '어묵', '향미유', '소고기', '떡류', '소스', '건면']",풀무원 떡볶이(쌀떡/밀떡/누들/고추장크림/양송이크림) 5봉 골라담기,"['떡류', '기타가공품', '소스', '소스류']"
4,프레시지 우삼겹 마라 떡볶이 2인분,"['소시지', '어묵', '향미유', '소고기', '떡류', '소스', '건면']",풀무원 전골 밀키트 4종(만두전골/어묵전골) 2개 골라담기,"['만두', '숙면', '혼합야채', '어묵', '소고기', '소스']"
5,프레시지 우삼겹 마라 떡볶이 2인분,"['소시지', '어묵', '향미유', '소고기', '떡류', '소스', '건면']",당일 제조하여 더욱 신선한 쿡솜씨 쓰촨마라탕,"['배추', '숙면', '어묵', '소고기', '목이버섯', '소스', '유바', ..."
6,프레시지 우삼겹 마라 떡볶이 2인분,"['소시지', '어묵', '향미유', '소고기', '떡류', '소스', '건면']",[신특한레시피] 어묵으로 감싼 만두전골 밀키트 (냉동),"['어묵', '표고버섯', '소스', '건고추', '양파', '대파']"
7,프레시지 우삼겹 마라 떡볶이 2인분,"['소시지', '어묵', '향미유', '소고기', '떡류', '소스', '건면']",풀무원 쌀 국물떡볶이 423.5g(2인분) x 4봉,"['떡류', '기타가공품', '소스류']"
8,프레시지 우삼겹 마라 떡볶이 2인분,"['소시지', '어묵', '향미유', '소고기', '떡류', '소스', '건면']",풀무원 밀 누들 국물떡볶이 423.5g(2인분) x 4봉,"['떡류', '기타가공품', '소스류']"
9,프레시지 우삼겹 마라 떡볶이 2인분,"['소시지', '어묵', '향미유', '소고기', '떡류', '소스', '건면']",당일 제조하여 더욱 신선한 쿡솜씨 안동찜닭,"['청양고추', '감자', '떡류', '소스', '당근', '건고추', '양파', ..."
